# Explore here

In [1]:
import os
import pandas as pd

# Lista de carpetas con datos diarios
carpetas = [
    '/workspaces/Finarosalina_proyecto_final/data/raw/Datos_diarios_2021',
    '/workspaces/Finarosalina_proyecto_final/data/raw/Datos_diarios_2022',
    '/workspaces/Finarosalina_proyecto_final/data/raw/Datos_diarios 2023'  # ojo con el espacio
]

dfs = []

for carpeta in carpetas:
    print(f"Procesando carpeta: {carpeta}")
    # Extraer el año del nombre de la carpeta
    # Por ejemplo, buscar los 4 dígitos de año en el string
    import re
    match = re.search(r'\b(20\d{2})\b', carpeta)
    year = match.group(1) if match else "unknown"
    
    for archivo in os.listdir(carpeta):
        if archivo.endswith('.csv'):
            ruta_csv = os.path.join(carpeta, archivo)
            print(f" Leyendo archivo: {archivo}")
            df_temp = pd.read_csv(ruta_csv, sep=';')  # ajusta separador si es necesario
            df_temp['Year'] = year
            dfs.append(df_temp)

# Combinar todos
df_combinado = pd.concat(dfs, ignore_index=True)
print(f"Datos combinados: {df_combinado.shape}")

# Guardar combinado
df_combinado.to_csv('/workspaces/Finarosalina_proyecto_final/data/processed/datos_combinados.csv', index=False, sep=';')
print("Archivo combinado guardado.")



Procesando carpeta: /workspaces/Finarosalina_proyecto_final/data/raw/Datos_diarios_2021


FileNotFoundError: [Errno 2] No such file or directory: '/workspaces/Finarosalina_proyecto_final/data/raw/Datos_diarios_2021'

Están dando problemas las descargas y lecturas de archivos, porque están esctructurados de forma diferente, por eso hacemos una prueba con menos datos. Ahora que va todo ok paso a leer los archivos en csv.

In [ ]:
def combinar_csv_rutas(ruta_base, archivo_salida):
    dfs = []
    # Recorrer todas las carpetas y subcarpetas
    for carpeta, _, archivos in os.walk(ruta_base):
        print(f"Procesando carpeta: {carpeta}")
        for archivo in archivos:
            if archivo.lower().endswith('.csv'):
                ruta_archivo = os.path.join(carpeta, archivo)
                print(f" Leyendo archivo: {archivo}")
                try:
                    df = pd.read_csv(ruta_archivo, sep=None, engine='python')  # Detectar separador automáticamente
                    dfs.append(df)
                except Exception as e:
                    print(f"  Error leyendo {archivo}: {e}")
    
    if dfs:
        df_combinado = pd.concat(dfs, ignore_index=True)
        print(f"Datos combinados: {df_combinado.shape}")
        df_combinado.to_csv(archivo_salida, index=False)
        print(f"Archivo combinado guardado en: {archivo_salida}")
    else:
        print("No se encontraron archivos CSV para combinar.")

# Uso:
ruta_base = "/workspaces/Finarosalina_proyecto_final/data/raw"
archivo_salida = "/workspaces/Finarosalina_proyecto_final/data/combined_air_quality_data.csv"

combinar_csv_rutas(ruta_base, archivo_salida)


Procesando carpeta: /workspaces/Finarosalina_proyecto_final/data/raw
Procesando carpeta: /workspaces/Finarosalina_proyecto_final/data/raw/diarios_2019_tcm30-513558
 Leyendo archivo: PM25_DD_2019.csv
 Leyendo archivo: As_DD_2019.csv
 Leyendo archivo: Cd_DD_2019.csv
 Leyendo archivo: PM10_DD_2019.csv
 Leyendo archivo: C6H6_DD_2019.csv
 Leyendo archivo: Pb_DD_2019.csv
 Leyendo archivo: BaP_DD_2019.csv
 Leyendo archivo: Ni_DD_2019.csv
Procesando carpeta: /workspaces/Finarosalina_proyecto_final/data/raw/Datos_diarios_2021
 Leyendo archivo: PM25_DD_2021.csv
 Leyendo archivo: BaP_DD_2021.csv
 Leyendo archivo: As_DD_2021.csv


 Leyendo archivo: Cd_DD_2021.csv
 Leyendo archivo: PM10_DD_2021.csv
 Leyendo archivo: Ni_DD_2021.csv
 Leyendo archivo: Pb_DD_2021.csv
Procesando carpeta: /workspaces/Finarosalina_proyecto_final/data/raw/Datos_diarios_2022
 Leyendo archivo: Ni_DD_2022.csv
 Leyendo archivo: PM2.5_DD_2022.csv
 Leyendo archivo: B(a)P_DD_2022.csv
 Leyendo archivo: Cd_DD_2022.csv
 Leyendo archivo: PM10_DD_2022.csv
 Leyendo archivo: C6H6_DD_2022.csv
 Leyendo archivo: Pb_DD_2022.csv
 Leyendo archivo: As_DD_2022.csv
Procesando carpeta: /workspaces/Finarosalina_proyecto_final/data/raw/2020_diarios_2020_tcm30-531263
 Leyendo archivo: B(a)P_DD_2020.csv
 Leyendo archivo: Pb_DD_2020.csv
 Leyendo archivo: PM10_DD_2020.csv
 Leyendo archivo: PM2.5_DD_2020.csv
 Leyendo archivo: Ni_DD_2020.csv
 Leyendo archivo: Cd_DD_2020.csv
 Leyendo archivo: As_DD_2020.csv
 Leyendo archivo: C6H6_DD_2020.csv
Procesando carpeta: /workspaces/Finarosalina_proyecto_final/data/raw/Datos_diarios 2023
 Leyendo archivo: C6H6_DD_2023.csv
 Leyend

In [ ]:
from functools import reduce
import re
# El archivo de 2013 estaba dañado por que lo que lo tengo que volver a subir y añadir al archivo ya unificado.
# Función para procesar CSV 2013 con sep=';'
def procesar_csv_2013(ruta_archivo):
    df = pd.read_csv(ruta_archivo, sep=';')
    id_vars = ['PROVINCIA','MUNICIPIO','ESTACION','MAGNITUD','PUNTO_MUESTREO','ANNO','MES']
    value_vars = [col for col in df.columns if col.startswith('D')]
    df_largo = df.melt(id_vars=id_vars, value_vars=value_vars, var_name='DIA', value_name='VALOR')
    df_largo['DIA'] = df_largo['DIA'].str.extract('D(\d+)').astype(int)
    df_largo['FECHA'] = pd.to_datetime(dict(year=df_largo['ANNO'], month=df_largo['MES'], day=df_largo['DIA']), errors='coerce')
    return df_largo

# Función para leer y combinar todos CSV 2013
def leer_y_combinar_2013(ruta_carpeta_2013):
    archivos_2013 = [f for f in os.listdir(ruta_carpeta_2013) if f.endswith('.csv')]
    dfs_2013 = []
    for archivo in archivos_2013:
        ruta = os.path.join(ruta_carpeta_2013, archivo)
        print(f"Leyendo {ruta}")
        df_largo = procesar_csv_2013(ruta)
        contaminante = archivo.split('_')[0].replace('B(a)P','BaP').replace('PM2.5','PM25').replace('.csv','')
        df_largo = df_largo.rename(columns={'VALOR': contaminante})

        merge_cols = ['PROVINCIA','MUNICIPIO','ESTACION','MAGNITUD','PUNTO_MUESTREO','ANNO','MES','DIA','FECHA']
        cols_para_merge = [c for c in merge_cols if c in df_largo.columns] + [contaminante]
        df_largo = df_largo[cols_para_merge]

        dfs_2013.append(df_largo)
    
    df_2013_combinado = reduce(
        lambda left, right: pd.merge(left, right, on=merge_cols, how='outer'), dfs_2013
    )
    return df_2013_combinado

# Función para procesar Excel 2014-2016 (formato ancho a largo)
def procesar_excel_2014_2016(ruta_archivo):
    df = pd.read_excel(ruta_archivo)
    id_vars = ['PROVINCIA', 'MUNICIPIO', 'ESTACION', 'MAGNITUD', 'PUNTO_MUESTREO', 'ANNO', 'MES']
    value_vars = [col for col in df.columns if re.match(r'D\d{2}', col)]
    
    if not value_vars:
        print(f"⚠️ No se encontraron columnas de días en {ruta_archivo}")
        return pd.DataFrame()
    
    df_largo = df.melt(id_vars=id_vars, value_vars=value_vars, var_name='DIA', value_name='VALOR')
    df_largo['DIA'] = df_largo['DIA'].str.extract(r'D(\d{2})').astype(int)
    df_largo['FECHA'] = pd.to_datetime(dict(year=df_largo['ANNO'], month=df_largo['MES'], day=df_largo['DIA']), errors='coerce')
    
    contaminante = os.path.basename(ruta_archivo).split('_')[0]
    contaminante = contaminante.replace('B(a)P','BaP').replace('PM2.5','PM25')
    df_largo = df_largo.rename(columns={'VALOR': contaminante})
    
    cols_requeridas = id_vars + ['DIA', 'FECHA', contaminante]
    df_largo = df_largo[cols_requeridas]
    
    return df_largo

# Función para leer y combinar todos Excel 2014-2016
def leer_excels_2014_2016(ruta_excel_base):
    dfs = []
    for carpeta, _, archivos in os.walk(ruta_excel_base):
        print(f"Procesando carpeta Excel: {carpeta}")
        for archivo in archivos:
            if archivo.endswith('.xlsx') or archivo.endswith('.xls'):
                ruta_archivo = os.path.join(carpeta, archivo)
                print(f" Leyendo archivo: {ruta_archivo}")
                df_procesado = procesar_excel_2014_2016(ruta_archivo)
                if not df_procesado.empty:
                    dfs.append(df_procesado)
    
    if dfs:
        df_excels = pd.concat(dfs, ignore_index=True)
        return df_excels
    else:
        print("No se encontraron archivos Excel válidos para procesar.")
        return pd.DataFrame()

# Función para extraer año del nombre archivo (por si no está en datos)
def extraer_anio_desde_nombre(nombre_archivo):
    match = re.search(r'(20\d{2})', nombre_archivo)
    if match:
        return int(match.group(1))
    return None

# --- EJECUCIÓN PRINCIPAL ---

ruta_carpeta_2013 = "/workspaces/Finarosalina_proyecto_final/data/raw/2013_datos_diarios"
ruta_excel_2014_2016 = "/workspaces/Finarosalina_proyecto_final/data/raw/excel_2014_2016"
ruta_combinado_actual = "/workspaces/Finarosalina_proyecto_final/data/combined_air_quality_data.csv"
ruta_salida_final = ruta_combinado_actual

print("Leyendo datos combinados existentes...")
df_existente = pd.read_csv(ruta_combinado_actual, sep=';', low_memory=False)

print("\nProcesando archivos CSV 2013...")
df_2013 = leer_y_combinar_2013(ruta_carpeta_2013)

print("\nProcesando archivos Excel 2014-2016...")
df_2014_2016 = leer_excels_2014_2016(ruta_excel_2014_2016)

# Asegurar que 'ANNO' y demás existan (si no, inferir)
for df in [df_2013, df_2014_2016]:
    if 'ANNO' not in df.columns or df['ANNO'].isnull().all():
        print("Atención: Año no detectado en algunos datos, intenta inferirlo")
        # Aquí podrías añadir lógica para inferir año si fuese necesario

# Columnas para merge y concatenar
merge_cols = ['PROVINCIA','MUNICIPIO','ESTACION','MAGNITUD','PUNTO_MUESTREO','ANNO','MES','DIA','FECHA']

def filtrar_columnas(df):
    contaminantes = [c for c in df.columns if c not in merge_cols]
    cols_necesarias = merge_cols + contaminantes
    return df[cols_necesarias]

df_existente = filtrar_columnas(df_existente)
df_2013 = filtrar_columnas(df_2013)
df_2014_2016 = filtrar_columnas(df_2014_2016)

print("\nConcatenando todos los datos...")
df_final = pd.concat([df_existente, df_2013, df_2014_2016], ignore_index=True)

print(f"\nGuardando archivo combinado actualizado en {ruta_salida_final} ...")
df_final.to_csv(ruta_salida_final, sep=';', index=False)

print("Proceso terminado con éxito.")


Leyendo datos combinados existentes...

Procesando archivos CSV 2013...
Leyendo /workspaces/Finarosalina_proyecto_final/data/raw/2013_datos_diarios/Pb_2013_DD.csv
Leyendo /workspaces/Finarosalina_proyecto_final/data/raw/2013_datos_diarios/Ni_2013_DD.csv
Leyendo /workspaces/Finarosalina_proyecto_final/data/raw/2013_datos_diarios/BaP_2013_DD.csv
Leyendo /workspaces/Finarosalina_proyecto_final/data/raw/2013_datos_diarios/Cd_2013_DD.csv
Leyendo /workspaces/Finarosalina_proyecto_final/data/raw/2013_datos_diarios/PM2.5_2013_DD.csv
Leyendo /workspaces/Finarosalina_proyecto_final/data/raw/2013_datos_diarios/As_2013_DD.csv
Leyendo /workspaces/Finarosalina_proyecto_final/data/raw/2013_datos_diarios/PM10_2013_DD.csv

Procesando archivos Excel 2014-2016...
Procesando carpeta Excel: /workspaces/Finarosalina_proyecto_final/data/raw/excel_2014_2016
Procesando carpeta Excel: /workspaces/Finarosalina_proyecto_final/data/raw/excel_2014_2016/2015_datos_Diarios
 Leyendo archivo: /workspaces/Finarosalina_p

 DataFrame limpio y cargado (más de 1.4 millones de filas) con las columnas importantes, y con muchas columnas de días D01–D31 y contaminantes como Pb, Ni, BaP, etc.

Para resumir un poco:

    La tabla tiene 47 columnas.

    Hay bastantes nulos en las columnas diarias (D01 a D31).

    Algunos contaminantes tienen valores NaN (por ejemplo Pb).

    Las columnas clave (PROVINCIA, MUNICIPIO, ESTACION, ANNO, MES, DIA, FECHA y CONTAMINANTES) están presentes y bien cargadas.

In [ ]:
# 1. Eliminar las columnas D01 a D31 si existen
cols_d = [f'D{i:02d}' for i in range(1, 32)]
cols_to_drop = [col for col in cols_d if col in df_final.columns]
df_final = df_final.drop(columns=cols_to_drop)

# 2. revisar si las columnas DIA y FECHA están y no tienen valores nulos
print(df_final[['DIA', 'FECHA']].info())

# 3. Guardar el DataFrame limpio en CSV
output_path = '/workspaces/Finarosalina_proyecto_final/data/cleaned_air_quality_data.csv'
df_final.to_csv(output_path, index=False)

print(f'DataFrame limpio guardado en: {output_path}')


<class 'pandas.core.frame.DataFrame'>
Index: 1450328 entries, 393449 to 1843776
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   DIA     1422435 non-null  float64
 1   FECHA   1396303 non-null  object 
dtypes: float64(1), object(1)
memory usage: 33.2+ MB
None
DataFrame limpio guardado en: /workspaces/Finarosalina_proyecto_final/data/cleaned_air_quality_data.csv


In [ ]:
df_final.tail()

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANNO,MES,DIA,FECHA,Pb,Ni,BaP,Cd,PM25,As,PM10
1843772,7.0,40.0,5.0,9.0,07040005_9_M,2016.0,10.0,31.0,2016-10-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1843773,7.0,40.0,5.0,9.0,07040005_9_M,2016.0,11.0,31.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1843774,7.0,40.0,5.0,9.0,07040005_9_M,2016.0,12.0,31.0,2016-12-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1843775,22.0,158.0,2.0,9.0,22158002_9_M,2016.0,1.0,31.0,2016-01-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1843776,22.0,158.0,2.0,9.0,22158002_9_M,2016.0,12.0,31.0,2016-12-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
print(df_final['MAGNITUD'].value_counts())


MAGNITUD
10.0    338690
19.0    202942
17.0    197322
28.0    197096
62.0    195406
9.0     190870
27.0    128002
Name: count, dtype: int64


In [ ]:
magnitud_dict = {
    9: 'PM2.5',
    10: 'PM10',
    17: 'As',
    19: 'Pb',
    27: 'BaP',
    28: 'Cd',
    62: 'Ni'
}


In [ ]:
df_final['CONTAMINANTE'] = df_final['MAGNITUD'].map(magnitud_dict)


In [ ]:
cols_contaminantes = ['Pb', 'Ni', 'BaP', 'Cd', 'PM25', 'As', 'PM10']
df_final = df_final.drop(columns=cols_contaminantes)
# es más facil identificarlos por su nombre, 

Como pesan tanto los archivos, he borrado los archivos originales, empleado para construir el data set, luego lo he limpiado y he guardado el data set agrupado. Como ya no están los archivos al volver a cargar el codespace, me da error por lo que volverá a cargar el data set final guardado.

In [1]:
import pandas as pd
df_airq=pd.read_csv('/workspaces/Finarosalina_proyecto_final/data/cleaned_air_quality_data.csv')

In [3]:
# https://www.ine.es/daco/daco42/codmun/cod_provincia.htm

codigos_provincias = {
    "01": "Alava",
    "02": "Albacete",
    "03": "Alicante",
    "04": "Almeria",
    "05": "Avila",
    "06": "Badajoz",
    "07": "Illes Balears",
    "08": "Barcelona",
    "09": "Burgos",
    "10": "Caceres",
    "11": "Cadiz",
    "12": "Castellon",
    "13": "Ciudad Real",
    "14": "Cordoba",
    "15": "A Coruna",
    "16": "Cuenca",
    "17": "Girona",
    "18": "Granada",
    "19": "Guadalajara",
    "20": "Gipuzkoa",
    "21": "Huelva",
    "22": "Huesca",
    "23": "Jaen",
    "24": "Leon",
    "25": "Lleida",
    "26": "La Rioja",
    "27": "Lugo",
    "28": "Madrid",
    "29": "Malaga",
    "30": "Murcia",
    "31": "Navarra",
    "32": "Ourense",
    "33": "Asturias",
    "34": "Palencia",
    "35": "Las Palmas",
    "36": "Pontevedra",
    "37": "Salamanca",
    "38": "Santa Cruz de Tenerife",
    "39": "Cantabria",
    "40": "Segovia",
    "41": "Sevilla",
    "42": "Soria",
    "43": "Tarragona",
    "44": "Teruel",
    "45": "Toledo",
    "46": "Valencia",
    "47": "Valladolid",
    "48": "Bizkaia",
    "49": "Zamora",
    "50": "Zaragoza",
    "51": "Ceuta",
    "52": "Melilla"
}


In [4]:
# Asegurar que los códigos de provincia están en formato de 2 dígitos como strings
df_airq['PROVINCIA'] = df_airq['PROVINCIA'].astype(int).astype(str).str.zfill(2)

df_airq['NOMBRE_PROVINCIA'] = df_airq['PROVINCIA'].map(codigos_provincias)


In [6]:
df_airq.head()

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANNO,MES,DIA,FECHA,Pb,Ni,BaP,Cd,PM25,As,PM10,NOMBRE_PROVINCIA
0,01,59.0,9.0,10.0,01059009_10_M,2013.0,1.0,1.0,2013-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alava
1,01,59.0,9.0,10.0,01059009_10_M,2013.0,1.0,2.0,2013-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alava
2,01,59.0,9.0,10.0,01059009_10_M,2013.0,1.0,3.0,2013-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alava
3,01,59.0,9.0,10.0,01059009_10_M,2013.0,1.0,4.0,2013-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alava
4,01,59.0,9.0,10.0,01059009_10_M,2013.0,1.0,5.0,2013-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alava


In [5]:
# Eliminar decimales falsos tipo '59.0' → '59', y luego convertir a int
df_airq['MUNICIPIO'] = df_airq['MUNICIPIO'].astype(str).str.replace('.0', '', regex=False).astype(int)

# Obtener y ordenar los códigos únicos
municipios_unicos = sorted(df_airq['MUNICIPIO'].unique())



In [6]:
# para sustituir los condigos de los municipios por su nombre necesito cargar los codigos de los municipios del INE
df_2013 = pd.read_excel('/workspaces/Finarosalina_proyecto_final/data/raw/13codmun.xls')
df_2023 = pd.read_excel('/workspaces/Finarosalina_proyecto_final/data/raw/diccionario23.xlsx')

df_2013.head()

,Relación de municipios y códigos por provincias a 1 de enero de 2013,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,CPRO,CMUN,DC,NOMBRE
1,01,001,4,Alegría-Dulantzi
2,01,002,9,Amurrio
3,01,049,3,Añana
4,01,003,5,Aramaio


In [7]:
df_2023.head()

,Relación de municipios y códigos por comunidades autónomas y provincias a 1 de enero de 2023,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,CODAUTO,CPRO,CMUN,DC,NOMBRE
1,16,01,051,3,Agurain/Salvatierra
2,16,01,001,4,Alegría-Dulantzi
3,16,01,002,9,Amurrio
4,16,01,049,3,Añana


In [8]:
df_2023.columns = df_2023.iloc[0]  # Asigna la primera fila como nombres de columnas
df_2023 = df_2023.drop(df_2023.index[0]).reset_index(drop=True)  # Elimina esa fila
df_2013.columns = df_2013.iloc[0]  # Asigna la primera fila como nombres de columnas
df_2013 = df_2013.drop(df_2013.index[0]).reset_index(drop=True)  # Elimina esa fila

In [11]:
# para quedarnos solo con los datos de las capitales de provincia necesito convertir los municipios nombre.
# a lo largo de los años los codigos de los municipios pueden haber cambiado de ahi que compare el primer y ultimo año

# Normalizar nombres columnas. Añado una col de nan a 2013 para que tengan el mismo numero de columnas, y poder cotejar las tablas.
import numpy as np 
df_2013['COL_NAN'] = np.nan

df_2013.columns = ['COL_NAN', 'CPRO', 'CMUN', 'DC', 'NOMBRE']
df_2023.columns = ['CODAUTO', 'CPRO', 'CMUN', 'DC', 'NOMBRE']

# claves únicas para códigos (con ceros para estandarizar)
df_2013['clave'] = df_2013['CPRO'].astype(str).str.zfill(2) + df_2013['CMUN'].astype(str).str.zfill(3)
df_2023['clave'] = df_2023['CPRO'].astype(str).str.zfill(2) + df_2023['CMUN'].astype(str).str.zfill(3)

# Detectar códigos que están en 2013 pero no en 2023 y viceversa
claves_2013 = set(df_2013['clave'])
claves_2023 = set(df_2023['clave'])

solo_2013 = claves_2013 - claves_2023
solo_2023 = claves_2023 - claves_2013

print(f'Municipios con código solo en 2013: {len(solo_2013)}')
print(f'Municipios con código solo en 2023: {len(solo_2023)}')

# Mostrar diferencias para inspección
print("Diferencias 2013:")
print(df_2013[df_2013['clave'].isin(solo_2013)][['clave', 'NOMBRE']])
print("Diferencias 2023:")
print(df_2023[df_2023['clave'].isin(solo_2023)][['clave', 'NOMBRE']])

# como los que han cambiado no son capitales de provincia, no  afecta para descodificar

Municipios con código solo en 2013: 2959
Municipios con código solo en 2023: 8131
Diferencias 2013:
       clave  NOMBRE
0     001004     NaN
1     002009     NaN
2     049003     NaN
3     003005     NaN
4     006006     NaN
...      ...     ...
8112  296007     NaN
8113  297003     NaN
8114  298009     NaN
8115  001003     NaN
8116  001008     NaN

[8117 rows x 2 columns]
Diferencias 2023:
      clave               NOMBRE
0     01051  Agurain/Salvatierra
1     01001     Alegría-Dulantzi
2     01002              Amurrio
3     01049                Añana
4     01003              Aramaio
...     ...                  ...
8126  50296            Zaida, La
8127  50297             Zaragoza
8128  50298                Zuera
8129  51001                Ceuta
8130  52001              Melilla

[8131 rows x 2 columns]


In [12]:
df_airq.head()

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANNO,MES,DIA,FECHA,Pb,Ni,BaP,Cd,PM25,As,PM10,NOMBRE_PROVINCIA
0,01,59,9.0,10.0,01059009_10_M,2013.0,1.0,1.0,2013-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alava
1,01,59,9.0,10.0,01059009_10_M,2013.0,1.0,2.0,2013-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alava
2,01,59,9.0,10.0,01059009_10_M,2013.0,1.0,3.0,2013-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alava
3,01,59,9.0,10.0,01059009_10_M,2013.0,1.0,4.0,2013-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alava
4,01,59,9.0,10.0,01059009_10_M,2013.0,1.0,5.0,2013-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alava


In [13]:
# Paso 1: Crear clave en df_airq
df_airq['clave'] = df_airq['PROVINCIA'].astype(int).astype(str).str.zfill(2) + df_airq['MUNICIPIO'].astype(int).astype(str).str.zfill(3)

# Paso 2: Diccionario de provincias y sus capitales (sin tildes ni caracteres especiales)
capitales_dict = {
    '01': 'Vitoria', '02': 'Albacete', '03': 'Alicante', '04': 'Almeria', '05': 'Avila',
    '06': 'Badajoz', '07': 'Palma', '08': 'Barcelona', '09': 'Burgos', '10': 'Caceres',
    '11': 'Cadiz', '12': 'Castellon', '13': 'Ciudad Real', '14': 'Cordoba',
    '15': 'Coruna', '16': 'Cuenca', '17': 'Girona', '18': 'Granada', '19': 'Guadalajara',
    '20': 'San Sebastian', '21': 'Huelva', '22': 'Huesca', '23': 'Jaen', '24': 'Leon',
    '25': 'Lleida', '26': 'Logrono', '27': 'Lugo', '28': 'Madrid', '29': 'Malaga',
    '30': 'Murcia', '31': 'Pamplona', '32': 'Ourense', '33': 'Oviedo', '34': 'Palencia',
    '35': 'Las Palmas', '36': 'Pontevedra', '37': 'Salamanca', '38': 'Santa Cruz de Tenerife',
    '39': 'Santander', '40': 'Segovia', '41': 'Sevilla', '42': 'Soria', '43': 'Tarragona',
    '44': 'Teruel', '45': 'Toledo', '46': 'Valencia', '47': 'Valladolid', '48': 'Bilbao',
    '49': 'Zamora', '50': 'Zaragoza', '51': 'Ceuta', '52': 'Melilla'
}

# Paso 3: Obtener claves de capitales a partir de df_2023
capitales_codigos = []

# Normalizar nombres en df_2023 para facilitar la búsqueda
df_2023['NOMBRE_NORM'] = (
    df_2023['NOMBRE']
    .str.normalize('NFKD')
    .str.encode('ascii', errors='ignore')
    .str.decode('utf-8')
    .str.lower()
)

for cpro, nombre_capital in capitales_dict.items():
    nombre_norm = nombre_capital.lower()
    match = df_2023[
        (df_2023['CPRO'] == cpro) &
        (df_2023['NOMBRE_NORM'].str.contains(nombre_norm, na=False))
    ]
    if not match.empty:
        capitales_codigos.append(match.iloc[0]['clave'])  # Tomamos el primer match
    else:
        print(f"⚠️ No se encontro la capital '{nombre_capital}' con codigo provincia {cpro}")

# Paso 4: Filtrar df_airq por esas claves
df_capitales = df_airq[df_airq['clave'].isin(capitales_codigos)].copy()


⚠️ No se encontro la capital 'Castellon' con codigo provincia 12
⚠️ No se encontro la capital 'Las Palmas' con codigo provincia 35


In [14]:
# Convertir a enteros por si hay decimales
df_airq['PROVINCIA'] = df_airq['PROVINCIA'].astype(int)
df_airq['MUNICIPIO'] = df_airq['MUNICIPIO'].astype(int)

# Ver si hay coincidencia con Castelló de la Plana
df_airq[
    (df_airq['PROVINCIA'] == 12) &
    (df_airq['MUNICIPIO'] == 40)
]

# Asegurarse de que los valores son enteros
df_airq['PROVINCIA'] = df_airq['PROVINCIA'].astype(int)
df_airq['MUNICIPIO'] = df_airq['MUNICIPIO'].astype(int)

# Buscar Las Palmas de Gran Canaria por clave (CPRO=35, CMUN=016)
df_airq[
    (df_airq['PROVINCIA'] == 35) &
    (df_airq['MUNICIPIO'] == 16)
]

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANNO,MES,DIA,FECHA,Pb,Ni,BaP,Cd,PM25,As,PM10,NOMBRE_PROVINCIA,clave


In [21]:
df_2023.head()

,CODAUTO,CPRO,CMUN,DC,NOMBRE,clave,NOMBRE_NORM
0,16,01,051,3,Agurain/Salvatierra,01051,agurain/salvatierra
1,16,01,001,4,Alegría-Dulantzi,01001,alegria-dulantzi
2,16,01,002,9,Amurrio,01002,amurrio
3,16,01,049,3,Añana,01049,anana
4,16,01,003,5,Aramaio,01003,aramaio


In [16]:
df_tenerife = df_airq[df_airq['PROVINCIA'] == '38']

print(f"Filas de la provincia Santa Cruz de Tenerife: {len(df_tenerife)}")
print(df_tenerife)

Filas de la provincia Santa Cruz de Tenerife: 0
Empty DataFrame
Columns: [PROVINCIA, MUNICIPIO, ESTACION, MAGNITUD, PUNTO_MUESTREO, ANNO, MES, DIA, FECHA, Pb, Ni, BaP, Cd, PM25, As, PM10, NOMBRE_PROVINCIA, clave]
Index: []


In [ ]:
# Ver todos los MUNICIPIOS únicos con PROVINCIA 35 (Las Palmas)
df_airq[df_airq['PROVINCIA'] == 35]['MUNICIPIO'].unique()

# no hay datos en Canarias


array([], dtype=int64)

In [ ]:
import numpy as np

# Obtener valores únicos ordenados
valores_unicos = np.sort(df_airq['PROVINCIA'].unique())

# Crear rango completo desde el mínimo al máximo
rango_completo = np.arange(valores_unicos.min(), valores_unicos.max() + 1)

# Encontrar valores faltantes
faltantes = np.setdiff1d(rango_completo, valores_unicos)

print("Valores únicos ordenados:")
print(valores_unicos)
print("\nValores faltantes en el rango:")
print(faltantes)

# no hay datos de Cuenca ni de Las Palmas


Valores únicos ordenados:
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 17 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 34 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50]

Valores faltantes en el rango:
[16 35]


In [20]:
df_airq.shape

(1450328, 18)

In [23]:
# Normalizar los nombres de capitales del diccionario (para que coincida con df_2023)
import unidecode

capitales_norm = {k: v.lower() for k, v in capitales_dict.items()}  # Ya sin tildes y en minúsculas

# Filtrar en df_2023 las filas que son capitales
capitales_codigos = []

for cpro, capital_norm in capitales_norm.items():
    fila = df_2023[(df_2023['CPRO'] == cpro) & (df_2023['NOMBRE_NORM'] == capital_norm)]
    if not fila.empty:
        capitales_codigos.append(fila.iloc[0])

df_capitales_codigos = pd.DataFrame(capitales_codigos)

# Formatear los códigos de provincia y municipio en df_airq
df_airq['CPRO'] = df_airq['PROVINCIA'].apply(lambda x: f"{int(x):02d}")
df_airq['CMUN'] = df_airq['MUNICIPIO'].apply(lambda x: f"{int(x):03d}")

# Merge para filtrar solo capitales
df_airq_capitales = pd.merge(
    df_airq,
    df_capitales_codigos[['CPRO', 'CMUN']],
    on=['CPRO', 'CMUN'],
    how='inner'
)

# Guardar resultado
df_airq_capitales.to_csv('df_airq_capitales.csv', index=False)

print("Dataset con solo capitales guardado.")


Dataset con solo capitales guardado.


In [35]:
df_airq_cap=pd.read_csv('/workspaces/Finarosalina_proyecto_final/data/raw/df_airq_capitales.csv')

In [39]:
df_airq_cap = df_airq_cap.drop(columns=['PROVINCIA', 'MUNICIPIO', 'MAGNITUD', 'CPRO', 'CMUN', 'PUNTO_MUESTREO'])

In [40]:
df_airq_cap.head()

,ESTACION,ANNO,MES,DIA,FECHA,Pb,Ni,BaP,Cd,PM25,As,PM10,NOMBRE_PROVINCIA,clave
0,1.0,2013.0,7.0,1.0,2013-07-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Albacete,2003
1,1.0,2013.0,7.0,2.0,2013-07-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Albacete,2003
2,1.0,2013.0,7.0,3.0,2013-07-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Albacete,2003
3,1.0,2013.0,7.0,4.0,2013-07-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Albacete,2003
4,1.0,2013.0,7.0,5.0,2013-07-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Albacete,2003


In [43]:
df_airq_cap['FECHA'] = pd.to_datetime(df_airq_cap['FECHA'], errors='coerce').dt.strftime('%Y-%m-%d')



In [41]:
df_airq_cap.shape

(408842, 14)

In [ ]:
import zipfile

#COMPRIMO EL ARCHIVO DE PARTIDA POR SI HACE FALTA MÁS TARDE YA QUE DE OTRA FORMA NO LO PUEDO DESCARGAR NI SUBIR

csv_path = "/workspaces/Finarosalina_proyecto_final/data/cleaned_air_quality_data.csv"
zip_path = "/workspaces/Finarosalina_proyecto_final/data/cleaned_air_quality_data.zip"

with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(csv_path, arcname="cleaned_air_quality_data.csv")


In [45]:
df_airq_cap.isna().sum()


ESTACION                 0
ANNO                     0
MES                      0
DIA                   7857
FECHA               331622
Pb                  383393
Ni                  384031
BaP                 395711
Cd                  383892
PM25                378570
As                  383809
PM10                369774
NOMBRE_PROVINCIA         0
clave                    0
dtype: int64

    Revisar porcentaje de nulos por columna (te ayuda a decidir si imputar o eliminar):

nulos_pct = df_final.isna().mean() * 100
print(nulos_pct.sort_values(ascending=False))

    Opcional: rellenar valores faltantes según el contexto (media, interpolación, etc.), por ejemplo para Pb:

df_final['Pb'] = df_final['Pb'].fillna(method='ffill')  # o con otra estrategia

    Confirmar tipos de datos para optimizar memoria y evitar errores:

print(df_final.dtypes)

    Exploración rápida: número de estaciones, años, contaminantes.

print("Estaciones únicas:", df_final['ESTACION'].nunique())
print("Años:", df_final['ANNO'].unique())
print("Contaminantes muestreados:", ['Pb', 'Ni', 'BaP', 'Cd', 'PM25', 'As', 'PM10'])

Unidad de medida
La unidad de medida depende del contaminante de que se trate:
μg/m3: dióxido de azufre (SO2), dióxido de nitrógeno (NO2), óxidos de nitrógeno (NOx), partículas en
suspensión de tamaño inferior a 10 μm (PM10), partículas en suspensión de tamaño inferior a 2,5 μm
(PM2,5), plomo (Pb), benceno (C6H6) y ozono (O3).
mg/m3: monóxido de carbono (CO).
ng/m3: arsénico (As), cadmio (Cd), níquel (Ni) y benzo(a)pireno (B(a)P).